In [10]:
from keras.models import Sequential

import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.models import Model


import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
import keras

from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import numpy as np
import os
import random 
from numpy import *
import theano
import pandas as pd
from keras.models import Model

import zipfile
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RandomRotation, Dense, Activation,RandomFlip, Dropout,MaxPooling2D, Flatten,Convolution2D,BatchNormalization, Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor



In [11]:

import shutil
working_dir = "/kaggle/working/"
for rootdir, dirs, files in os.walk(working_dir):
    for subdir in dirs:
        shutil.rmtree(subdir)

In [12]:
working_dir = '/kaggle/working/'
local_zip = '../input/dogs-vs-cats-redux-kernels-edition/train.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall(working_dir)
zip_ref.close()

In [13]:
# Creating folders for both the classes of cats and dogs in the directory
source = '/kaggle/working/train/'

dog_photos = '/kaggle/working/train/dogs/'
cats_photo = '/kaggle/working/train/cats/'
os.mkdir(dog_photos)
os.mkdir(cats_photo)

check the naming pattern of our files with the follwing command:

In [14]:
# Source: https://www.kaggle.com/code/mariateplinskaya/cats-dogs-cnn-2
train_set = os.listdir(source)
  
# print(len(allfiles))
for f in train_set:
  if os.path.isfile(source + f):
      if 'cat' in f:
        shutil.move(source + f, cats_photo + f)
      if 'dog' in f:
        shutil.move(source + f, dog_photos + f)


In [15]:

# Rescaling training data to 0-1 pixels
train_datagen = ImageDataGenerator(rescale=1./255)
# train_dir = "/content/drive/MyDrive/marksheets/train/"
train_dir = source
train_generator = train_datagen.flow_from_directory(
        train_dir,
        # Images will be resized to 100X100
        target_size=(100, 100),
        batch_size=500,
        class_mode='categorical')

In [16]:
print(train_generator.class_indices)

We can see train generator elements how does it look like. Lets see the first element of the train_generator.It will return a tuple of 32 images and 32 labels. Because our each training batch containes 32 images in it as specified by variable *train_batch = 32*

In [17]:
print(train_generator[0][0].shape)
print(train_generator[0][1].shape)

decimal value in 546.8 i.e. 0.8 shows there were 28 pictures left in the end, and our final batch has 28 pictures in it, whereas all other batches from 0 ~ 545 have 32 pictures and respective labels in it. likwise you can claculate batches in validation dataset.

In [18]:
# source: https://www.kaggle.com/code/cvtrongduc/cat-vs-dog-with-transfer-learning

my_base_model = keras.applications.DenseNet201(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(100, 100, 3),
    include_top=False,
)  


my_base_model.trainable = False

my_inputs = keras.Input(shape=(100, 100, 3))


x = my_base_model(my_inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(64, activation='relu')(x)
x = keras.layers.Dense(32, activation='relu')(x)
my_outputs = keras.layers.Dense(1, activation='sigmoid')(x)
my_model = keras.Model(my_inputs,my_outputs)

my_model.summary()

In [19]:
my_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=['accuracy']
)
early_stopping  = keras.callbacks.EarlyStopping(min_delta=0.001, patience=5)

epochs = 10
my_model.fit(train_generator, 
             epochs=epochs, 
             #validation_data=validation_generator,
            callbacks=[early_stopping])

In [20]:
# TRIAL MODEL 1
# nb_epochs = 10
# img_channel = 3
# nb_filters = 32
# nb_pool = 2
# nb_conv = 3
# #Pg 461 Book hands on machine learning
# model = keras.models.Sequential([
# keras.layers.Conv2D(64, 7, activation="relu", padding="same",
# input_shape=[100, 100, 3]),
# keras.layers.MaxPooling2D(2),
# keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
# keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
# keras.layers.MaxPooling2D(2),
# keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
# keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
# keras.layers.MaxPooling2D(2),
# keras.layers.Flatten(),
# keras.layers.Dense(128, activation="relu"),
# keras.layers.Dropout(0.5),
# keras.layers.Dense(64, activation="relu"),
# keras.layers.Dropout(0.5),
# keras.layers.Dense(1, activation="sigmoid")
# ])

In [21]:
# TRIAL MODEL 2
# nb_epochs = 10
# img_channel = 3
# nb_filters = 32
# nb_pool = 2
# nb_conv = 3
# # https://www.analyticsvidhya.com/blog/2021/01/image-classification-using-convolutional-neural-networks-a-step-by-step-guide/
# model = tf.keras.Sequential([
#     tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu, input_shape=(100, 100, 3)),
#     tf.keras.layers.MaxPooling2D((2, 2), strides=2),
#     tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu),
#     tf.keras.layers.MaxPooling2D((2, 2), strides=2),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(20, activation=tf.nn.relu),
#     tf.keras.layers.Dense(2,  activation=tf.nn.softmax)
# ])


Making predictions

In [22]:
working_dir = '/kaggle/working/'
local_zip_test = '../input/dogs-vs-cats-redux-kernels-edition/test.zip'
zip_ref = zipfile.ZipFile(local_zip_test, 'r')
zip_ref.extractall(working_dir)
zip_ref.close()

In [23]:
test_directory = '/kaggle/working/test/'


x_test = keras.utils.image_dataset_from_directory(
  test_directory,
  seed=68,
  labels=None,
  image_size=(100, 100),
  batch_size=500,
  interpolation='nearest',
  shuffle=False)


In [24]:
file_paths = os.listdir(test_directory)
file_path = []
for i in file_paths:
  j = i[:len(i)-4]
  file_path.append(int(j))


print (file_path)

In [25]:
y_pred = my_model.predict(x_test)

In [26]:
print(y_pred)

In [27]:
prob = []
for i in y_pred:
    prob.append(i[0])
print (len(prob))
print (prob)

In [28]:
e_d = {'id':file_path , 'label': prob }
df = pd.DataFrame(e_d)
df = df.sort_values(by=['id'], ascending = [True])

In [34]:
df.to_csv("/kaggle/working/MSBA.section2.garg0119.csv", index = False)